Test Cases for Fluxonium circuit :
* anHarmonicity flatness across flux spectrum
* analytically related to Ej

In [1]:
from DiSuQ.Torch import models
from DiSuQ.Torch.optimization import OrderingOptimization,PolynomialOptimization
from DiSuQ.Torch.optimization import lossAnharmonicity,lossTransitionFlatness
from torch import tensor
from numpy import arange,linspace
from DiSuQ.utils import plotCompare
from torch import set_num_threads
set_num_threads(32)

* initial circuit complies with oscillator basis
* spectrum is invariant with higher truncation

In [2]:
basis = {'O':[512],'I':[],'J':[]}; rep = 'K'
#basis = [512]; rep = 'Q'
circuit = models.fluxonium(basis,El=10.,Ec=100.,Ej=50,sparse=False)
print(circuit.circuitComponents())

{'Cap': 99.99999237060547, 'JJ': 50.000003814697266, 'I': 9.999999046325684}


In [3]:
flux_range = tensor(linspace(0,1,32,endpoint=True))

#### pre-optimization Fluxonium

In [4]:
flux_profile = [[flux] for flux in flux_range]
flux_point = ('I')

In [5]:
H_LC = circuit.kermanHamiltonianLC()
H_J = circuit.kermanHamiltonianJosephson

In [6]:
#H_LC = circuit.chargeHamiltonianLC()
#H_J = circuit.josephsonCharge

In [7]:
pre_E0,pre_Ex = circuit.spectrumManifold(flux_point,flux_profile,H_LC,H_J,excitation=[1,2])

In [8]:
pre_Ex = pre_Ex.detach().numpy()

In [48]:
import importlib
import DiSuQ.utils
importlib.reload(DiSuQ.utils)
from DiSuQ.utils import plotCompare

In [49]:
plotCompare(flux_range,{'Ist Excitation':pre_Ex[0],'IInd Excitation':pre_Ex[1],'Anharmonicity':pre_Ex[1]-2*pre_Ex[0]},
            'Fluxonium-Excitation','external flux','E(GHz)',export='pdf')

#### optimization

In [10]:
flux_profile = [{'I':flux} for flux in flux_range]

In [11]:
optim = OrderingOptimization(circuit,representation=rep)
dLogs,dParams,dCircuit = optim.optimization(lossTransitionFlatness,flux_profile,iterations=50,lr=.0005)

In [12]:
plotCompare(dLogs.index,dLogs,'Optimizing Fluxonium','iteration',export='pdf')

In [13]:
plotCompare(dCircuit.index,dCircuit,None,"iteration","energy",export='pdf')

Analytically the solution is $JJ=0$ \
However, the optimization stagnates asymptotically \
and therefore gradient descent algorithm improper

#### post-optimization Fluxonium

In [14]:
flux_profile = [[flux] for flux in flux_range]
flux_point = ('I')

In [15]:
H_LC = circuit.kermanHamiltonianLC()
H_J = circuit.kermanHamiltonianJosephson

In [16]:
post_E0,post_Ex = circuit.spectrumManifold(flux_point,flux_profile,H_LC,H_J,excitation=[1,2])

In [17]:
post_Ex = post_Ex.detach().numpy()

In [18]:
plotCompare(flux_range,{'Ist Excitation':post_Ex[0],'IInd Excitation':post_Ex[1],'Anharmonicity':post_Ex[1]-2*post_Ex[0]},
            'Fluxonium-Excitation','external flux','E(GHz)',export='pdf')

In [19]:
print(circuit.circuitComponents())

{'Cap': 114.05826568603516, 'JJ': 5.575027942657471, 'I': 8.406220436096191}


## Circuit Comparison

In [20]:
plotCompare(flux_range,{'post':post_Ex[1]-2*post_Ex[0],'pre':pre_Ex[1]-2*pre_Ex[0]},
            'Fluxonium-Anharmonicity','external flux','E(GHz)',export='pdf')